In [ ]:
# --- CELL 1: DUAL INGESTION (REAL + SYNTHETIC) ---
!pip install spotipy faker --quiet

import os
import json
import datetime
import random
import shutil
from google.colab import drive, userdata
import spotipy
from spotipy.oauth2 import SpotifyOAuth

# 1. Setup Paths & Cleanup
drive.mount('/content/drive')
base_path = "/content/drive/MyDrive/Spotify_Lakehouse"
bronze_path = f"{base_path}/bronze"
silver_path = f"{base_path}/silver"

# Clear Bronze to prevent duplicates
if os.path.exists(bronze_path):
    shutil.rmtree(bronze_path)
os.makedirs(bronze_path)
os.makedirs(silver_path, exist_ok=True)

# 2. Auth
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id=userdata.get('SPOTIFY_CLIENT_ID'),
    client_secret=userdata.get('SPOTIFY_CLIENT_SECRET'),
    redirect_uri="http://127.0.0.1:8888/callback",
    scope="user-read-recently-played",
    open_browser=False
))

# --- GENERATOR 1: SYNTHETIC DATA ---
print("Generating Synthetic Data...")
mock_playlist = [
    ("The Weeknd", "Blinding Lights", "After Hours"),
    ("Taylor Swift", "Cruel Summer", "Lover"),
    ("Arijit Singh", "Kesariya", "Brahmastra"),
    ("Drake", "God's Plan", "Scorpion"),
    ("Queen", "Bohemian Rhapsody", "A Night at the Opera"),
    ("Coldplay", "Yellow", "Parachutes")
]
fake_items = []
current_time = datetime.datetime.now()
for _ in range(50):
    artist, track, album = random.choice(mock_playlist)
    minutes_ago = random.randint(0, 1440)
    played_at = (current_time - datetime.timedelta(minutes=minutes_ago)).isoformat() + "Z"
    fake_items.append({
        "played_at": played_at,
        "track": {
            "name": track, "duration_ms": 200000, "popularity": 80,
            "album": {"name": album}, "artists": [{"name": artist}]
        }
    })

# Save Synthetic File
with open(f"{bronze_path}/synthetic.json", 'w') as f:
    json.dump({"items": fake_items}, f)
print("Synthetic Data Saved.")

# --- GENERATOR 2: REAL DATA ---
print("Fetching Real Data...")
try:
    results = sp.current_user_recently_played(limit=50)
    if len(results['items']) > 0:
        with open(f"{bronze_path}/real.json", 'w') as f:
            json.dump(results, f)
        print(f"Real Data Saved ({len(results['items'])} tracks).")
    else:
        print("Real Data is empty (Spotify API returned 0). Only Synthetic available.")
except Exception as e:
    print(f"Real Data Error: {e}")

print("Ingestion Complete.")

In [ ]:
# --- CELL 2: PYSPARK ETL (TAGGING SOURCES) ---
!pip install pyspark -q
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, from_utc_timestamp, hour, when, lit
from pyspark.sql.types import StructType, StructField, StringType, LongType, ArrayType
import os

spark = SparkSession.builder.appName("SpotifyLakehouse").getOrCreate()

# 1. Define Schema
track_schema = StructType([
    StructField("items", ArrayType(
        StructType([
            StructField("played_at", StringType(), True),
            StructField("track", StructType([
                StructField("name", StringType(), True),
                StructField("artists", ArrayType(StructType([StructField("name", StringType(), True)])), True),
                StructField("album", StructType([StructField("name", StringType(), True)]), True)
            ]), True)
        ])
    ), True)
])

# 2. Function to Process a specific file and add a Tag
def process_file(filename, source_tag):
    full_path = f"{bronze_path}/{filename}"
    if not os.path.exists(full_path):
        return None

    raw = spark.read.schema(track_schema).option("multiline", "true").json(full_path)

    return raw.select(explode("items").alias("item")) \
        .select(
            col("item.track.name").alias("track_name"),
            col("item.track.artists")[0]["name"].alias("artist_name"),
            col("item.played_at").alias("played_at_utc")
        ) \
        .withColumn("played_at_ist", from_utc_timestamp(col("played_at_utc"), "IST")) \
        .withColumn("hour", hour(col("played_at_ist"))) \
        .withColumn("time_period",
            when((col("hour") >= 5) & (col("hour") < 12), "Morning 🌅")
            .when((col("hour") >= 12) & (col("hour") < 17), "Afternoon ☀️")
            .when((col("hour") >= 17) & (col("hour") < 21), "Evening 🌆")
            .otherwise("Night 🌙")
        ) \
        .withColumn("data_source", lit(source_tag)) # <--- THIS IS THE MAGIC TAG

# 3. Process Both
df_syn = process_file("synthetic.json", "Synthetic Data")
df_real = process_file("real.json", "Real Spotify Data")

# 4. Combine (Union)
if df_real is not None:
    final_df = df_syn.union(df_real)
else:
    final_df = df_syn

# 5. Write to Silver
final_df.write.mode("overwrite").parquet(f"{silver_path}/tracks.parquet")
print("ETL Success: Combined Data saved.")
final_df.groupBy("data_source").count().show()

In [ ]:
# --- CELL 3: APP WITH TOGGLE (NO CACHE) ---
app_code = """
import streamlit as st
import pandas as pd
import plotly.express as px
import glob

st.set_page_config(page_title="Spotify Lakehouse", layout="wide")

# --- SIDEBAR CONTROLS ---
st.sidebar.title("🎛️ Controls")
st.sidebar.markdown("Use this switch to compare Real vs. Simulated data.")

# 1. Load Data (CACHE REMOVED)
# We removed @st.cache_data so it re-reads the file every time you refresh.
def load_data():
    path = "/content/drive/MyDrive/Spotify_Lakehouse/silver/tracks.parquet"
    if not glob.glob(f"{path}/*.parquet"):
        return pd.DataFrame()
    return pd.read_parquet(path, engine='pyarrow')

df_all = load_data()

if not df_all.empty:
    # 2. THE TOGGLE SWITCH
    available_sources = df_all['data_source'].unique()

    # Default to 'Real' if available
    index = 0
    if "Real Spotify Data" in available_sources:
        index = list(available_sources).index("Real Spotify Data")

    source_selection = st.sidebar.radio(
        "Select Data Source:",
        available_sources,
        index=index
    )

    # 3. FILTER DATA
    df = df_all[df_all['data_source'] == source_selection]

    # --- DASHBOARD UI ---
    st.title(f"🎧 Dashboard: {source_selection}")

    # Metrics
    c1, c2, c3 = st.columns(3)
    c1.metric("Tracks Analyzed", len(df))
    c2.metric("Unique Artists", df['artist_name'].nunique())
    top_artist = df['artist_name'].mode()[0] if not df.empty else "N/A"
    c3.metric("Top Artist", top_artist)

    st.divider()

    # Charts
    col1, col2 = st.columns(2)

    with col1:
        st.subheader("🎤 Top Artists")
        if not df.empty:
            counts = df['artist_name'].value_counts().head(10).reset_index()
            counts.columns = ['Artist', 'Count']
            fig = px.bar(counts, x='Count', y='Artist', orientation='h', color='Count')
            st.plotly_chart(fig, use_container_width=True)

    with col2:
        st.subheader("🕒 Listening Time")
        if not df.empty:
            t_counts = df['time_period'].value_counts().reset_index()
            t_counts.columns = ['Period', 'Count']
            fig2 = px.pie(t_counts, values='Count', names='Period', hole=0.4)
            st.plotly_chart(fig2, use_container_width=True)

    st.subheader("📄 Raw Data")
    st.dataframe(df.head(10), use_container_width=True)

else:
    st.error("No data found. Run ETL first.")
"""

with open("app.py", "w") as f:
    f.write(app_code)
print("App updated (Cache Disabled).")

In [ ]:
# --- CELL 4: LAUNCH ---
!pip install streamlit pyngrok plotly pandas pyarrow -q
from pyngrok import ngrok
from google.colab import userdata
import subprocess
import time

# Auth
try:
    ngrok.set_auth_token(userdata.get('NGROK_AUTH_TOKEN'))
except:
    print("Check your Ngrok Token!")

# Kill old tunnels
ngrok.kill()

# Run
subprocess.Popen(['streamlit', 'run', 'app.py'])
time.sleep(5)

# Open Tunnel
try:
    public_url = ngrok.connect(8501).public_url
    print(f"DASHBOARD LIVE: {public_url}")
except Exception as e:
    print(f"Tunnel Error: {e}")